In [1]:
from jqdata import *
import numpy as np
import pandas as pd
import pickle
################################################取期权#########################################################    
#######起始日期
start_date_option = '2015-02-09'
######终止日期
end_date_option = '2023-02-24'
######标的https://docs.qq.com/sheet/DTGZqRE5UbWlla2FH?tab=eqwbzr
# 上证50ETF期权， 510050.XSHG,上市时间为2015-02-09，从2015-05-01开始取数据,实际上2017-01-03号才开始有数据data_list[410:]
# 沪深300ETF期权（上海交易所），510300.XSHG，上市时间为2019-12-23
# 沪深300ETF期权（深圳交易所），159919，上市时间为2019-12-23
# 沪深300股指期货期权，IO，看涨期权：IO合约月份-C-行权价格，如IO2302-C-3350，看跌期权：IO合约月份-P-行权价格
# 中证1000股指期货期权，MO

underlying_asset_name = '50ETF'
option_file_name = 'option_contracts_{}_({}to{}).pkl'.format(underlying_asset_name,
                                                             start_date_option,
                                                             end_date_option)
option_folder_name = 'contracts_list'
option_path = option_folder_name + '/' + option_file_name

with open (option_path,'rb') as op_f:
    option_data_raw = pickle.load(op_f)
# data_list = option_data_raw[410:-16]# 2017-01-03开始 到2022-02-01
data_list = option_data_raw[:-1]#2015-02-09 to 2023-02-24
########################取ETF数据########################################################
start_date_etf = '2015-01-01'
end_date_etf = '2023-02-24'

# 上证50etf 2005-03-23上市
etf_code = '510050.XSHG'
######采样频率/min
freq = 1

etf_folder_name = 'bar_data'
etf_file_name = 'etf{}_{}min({}to{}).pkl'.format(etf_code,freq,
                                                 start_date_etf,
                                                 end_date_etf)
etf_path = etf_folder_name + '/' + etf_file_name
with open (etf_path,'rb') as etf_f:
    etf_raw_data = pickle.load(etf_f)
etf_bar_list = etf_raw_data[26:]
# etf_bar_list = etf_raw_data[411:-15]
########################################取移仓换月数据###############################################
# IH_adjusted = pd.read_csv('my_data/IH_data.csv')
##################################################################################################



In [3]:
def get_one_code_data(one_code_arr, date):
    datetime = str(date)+' 15:00:00' # 取数据的时间
    bar_etf = get_bars('510050.XSHG', 1, unit='30m',
                      fields=('close'),
                      include_now=True, end_dt=datetime, fq_ref_date=None, df=False)#include_now = True取当天收盘，False收盘前半小时
    one_code_arr[2] = bar_etf[0][0]

    q_gamma = query(opt.OPT_RISK_INDICATOR).filter(opt.OPT_RISK_INDICATOR.code == one_code_arr[0],
                                                   opt.OPT_RISK_INDICATOR.date == date)
    gamma = opt.run_query(q_gamma)['gamma']
    if (len(gamma) != 0):
        one_code_arr[4] = gamma[0]
    else:
        one_code_arr[4] = np.NaN
        
    q_op = query(opt.OPT_DAILY_PRICE).filter(opt.OPT_DAILY_PRICE.code==one_code_arr[0],
                                         opt.OPT_DAILY_PRICE.date==date)
    open_interest = opt.run_query(q_op)['position']
    if (len(open_interest) != 0):
        one_code_arr[5] = open_interest[0]
    else:
        one_code_arr[5] = np.NaN

In [4]:


def calc_daily_NGE(code_daily):
    date = code_daily[0] # current date
    code_temp = code_daily[1]
    code_arr = np.insert(code_temp,[2,2,2,2],np.NaN, axis=1)

    if type(code_daily[1]) is np.ndarray:    
        for _, one_code_arr in enumerate(code_arr):
            get_one_code_data(one_code_arr, date)
        code_arr[:,3] = code_arr[:,2].astype('float64') / code_arr[:,1].astype('float64') #alpha
        # code_arr(code,strike,close,alpha,gamma,open_interest,expiration_date,call or put, adjusted?)
        call_option = code_arr[code_arr[:,7]=='c']
        put_option = code_arr[code_arr[:,7]=='p']

    #     deep out of money(DOTM) alpha<0.955
    #     out of the money(OTM) 0.955<alpha<0.985
    #     at the money(ATM) 0.985<alpha<1.015
    #     in the money(ITM) 1.015<alpha<1.045
    #     deep in the money(DITM) 1.045<alpha
        call_DOTM = call_option[call_option[:,3] < 0.955]
        put_DITM = put_option[put_option[:,3] < 0.955]

        call_DITM = call_option[call_option[:,3] > 1.045]
        put_DOTM = put_option[put_option[:,3] > 1.045]

        call_OTM = call_option[np.logical_and(call_option[:,3]>=0.955, call_option[:,3]<0.985)]
        put_ITM  = put_option[np.logical_and(put_option[:,3]>=0.955, put_option[:,3]<0.985)]

        call_ITM = call_option[np.logical_and(call_option[:,3]>=1.015, call_option[:,3]<1.045)]
        put_OTM  = put_option[np.logical_and(put_option[:,3]>=1.015, put_option[:,3]<1.045)]

        call_ATM = call_option[np.logical_and(call_option[:,3]>=0.985, call_option[:,3]<1.015)]
        put_ATM  = put_option[np.logical_and(put_option[:,3]>=0.985, put_option[:,3]<1.015)]

        NGE_all = call_option[:,4].astype('float64')@call_option[:,5].astype('float64') - \
                  put_option[:,4].astype('float64')@put_option[:,5].astype('float64') 

        NGE_DITM = call_DITM[:,4].astype('float64')@call_DITM[:,5].astype('float64') - \
                   put_DITM[:,4].astype('float64')@put_DITM[:,5].astype('float64') 

        NGE_DOTM = call_DOTM[:,4].astype('float64')@call_DOTM[:,5].astype('float64') - \
                   put_DOTM[:,4].astype('float64')@put_DOTM[:,5].astype('float64') 

        NGE_ITM = call_ITM[:,4].astype('float64')@call_ITM[:,5].astype('float64') - \
                  put_ITM[:,4].astype('float64')@put_ITM[:,5].astype('float64') 

        NGE_OTM = call_OTM[:,4].astype('float64')@call_OTM[:,5].astype('float64') - \
                  put_OTM[:,4].astype('float64')@put_OTM[:,5].astype('float64') 

        NGE_ATM = call_ATM[:,4].astype('float64')@call_ATM[:,5].astype('float64') - \
                  put_ATM[:,4].astype('float64')@put_ATM[:,5].astype('float64') 

        NGE = (date, NGE_all, NGE_DITM, NGE_ITM, NGE_ATM, NGE_OTM, NGE_DOTM)
    else:
        NGE = (date, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN )
    return NGE
    


In [5]:
NGE_list = []
t=1
# code_daily = data_list[t]
# index_data_daily = index_data_raw[t]
day_num = len(data_list)
start = time.perf_counter()
for _, code_daily in enumerate(data_list):
#     try:
    NGE_list.append(calc_daily_NGE(code_daily))
#     except:
#         print('something wrong')
#         NGE_list.append(np.NaN)
    ###############进度条#################
    
    c = (t / day_num) * 100
    k = round(c)
    a = "*" * k
    b = "." * (100 - k)
    dur = time.perf_counter() - start
    t+=1
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c,a,b,dur),end = "")

100%[****************************************************************************************************->]8249.76s

In [7]:
NGE_df.to_csv('NGE_df.csv')

In [8]:
pd.read_csv('NGE_df.csv')

,Unnamed: 0,date,NGE_all,NGE_DITM,NGE_ITM,NGE_ATM,NGE_OTM,NGE_DOTM
0,0,2015-02-09,2007.561,1276.190,431.942,728.125,625.674,-1054.370
1,1,2015-02-10,2347.536,3286.935,694.467,1121.857,-161.985,-2593.738
2,2,2015-02-11,3415.058,4559.772,396.245,1781.216,955.391,-4277.566
3,3,2015-02-12,6736.921,4823.007,536.384,3252.744,1630.657,-3505.871
4,4,2015-02-13,2786.476,6324.830,288.774,1339.006,1062.347,-6228.481
5,5,2015-02-16,4345.334,6963.756,513.605,1398.746,1819.283,-6350.056
6,6,2015-02-17,6234.108,8687.209,-1463.527,1428.161,7392.923,-9810.658
7,7,2015-02-25,7652.045,6547.251,374.925,3066.116,440.495,-2776.742
8,8,2015-02-26,6357.382,5575.375,6630.226,3102.438,-147.096,-8803.561
9,9,2015-02-27,9965.720,8356.500,6535.166,3295.480,2321.667,-10543.093
